In [1]:
import sys
sys.path.append( '../../' )

In [2]:
import numpy as np
import cs_vqe as c
import ast
import os
from copy import deepcopy

In [3]:
from openfermion import qubit_operator_sparse
from quchem.Misc_functions import conversion_scripts as conv_scr
from openfermion.ops import QubitOperator

In [4]:
# with open("hamiltonians.txt", 'r') as input_file:
#     hamiltonians = ast.literal_eval(input_file.read())
    
cwd = os.getcwd()
working_dir = os.path.dirname(cwd)

Analysis_dir = os.path.join(working_dir, 'Analysis')

data_dir = os.path.join(working_dir, 'data')
data_hamiltonians_file = os.path.join(data_dir, 'hamiltonians.txt')


with open(data_hamiltonians_file, 'r') as input_file:
    hamiltonians = ast.literal_eval(input_file.read())

In [5]:
for key in hamiltonians.keys():
    print(f"{key: <25}     n_qubits:  {hamiltonians[key][1]:<5.0f}")

H2-S1_STO-3G_singlet          n_qubits:  18   
C1-O1_STO-3G_singlet          n_qubits:  16   
H1-Cl1_STO-3G_singlet         n_qubits:  16   
H1-Na1_STO-3G_singlet         n_qubits:  16   
H2-Mg1_STO-3G_singlet         n_qubits:  17   
H1-F1_3-21G_singlet           n_qubits:  18   
H1-Li1_3-21G_singlet          n_qubits:  18   
Be1_STO-3G_singlet            n_qubits:  5    
H1-F1_STO-3G_singlet          n_qubits:  8    
H1-Li1_STO-3G_singlet         n_qubits:  8    
Ar1_STO-3G_singlet            n_qubits:  13   
F2_STO-3G_singlet             n_qubits:  15   
H1-O1_STO-3G_singlet          n_qubits:  8    
H2-Be1_STO-3G_singlet         n_qubits:  9    
H2-O1_STO-3G_singlet          n_qubits:  10   
H2_3-21G_singlet              n_qubits:  5    
H2_6-31G_singlet              n_qubits:  5    
H3-N1_STO-3G_singlet          n_qubits:  13   
H4-C1_STO-3G_singlet          n_qubits:  14   
Mg1_STO-3G_singlet            n_qubits:  13   
N2_STO-3G_singlet             n_qubits:  15   
Ne1_STO-3G_si

In [6]:
# mol_key = 'H2_6-31G_singlet'  
# mol_key ='H2-O1_STO-3G_singlet'
# mol_key = 'H1-O1_STO-3G_singlet'
mol_key = 'H2-O1_STO-3G_singlet'


# currently index 2 is contextual part
# ''''''''''''''''3 is NON contextual part

# join together for full Hamiltonian:
ham = hamiltonians[mol_key][2]
ham.update(hamiltonians[mol_key][3]) # full H 
ham

{'IIIIIIIIII': -55.226577416214326,
 'IIIIIIIIIZ': 12.411397933347452,
 'IIIIIIIIYY': -0.007573527729508396,
 'IIIIIIIIZI': 12.411397933347452,
 'IIIIIIIIZZ': 1.1862777224861802,
 'IIIIIIIXIX': -0.10460420805448783,
 'IIIIIIIXXI': 0.0020603107584759067,
 'IIIIIIIXZX': -0.12499935118669338,
 'IIIIIIIYIY': -0.10460420805448783,
 'IIIIIIIYZY': -0.12499935118669338,
 'IIIIIIIZII': 1.6496131029811243,
 'IIIIIIIZIZ': 0.23693603049649825,
 'IIIIIIIZZI': 0.2515869355056082,
 'IIIIIIXIXI': -0.0033258799919286815,
 'IIIIIIXXYY': -0.014650905009109934,
 'IIIIIIXYYX': 0.014650905009109934,
 'IIIIIIXZXI': -0.12499935118669338,
 'IIIIIIXZXZ': -0.10460420805448783,
 'IIIIIIYIYI': -0.0033258799919286815,
 'IIIIIIYXXY': 0.014650905009109934,
 'IIIIIIYYII': -0.025296526569690943,
 'IIIIIIYYXX': -0.014650905009109934,
 'IIIIIIYZYI': -0.12499935118669338,
 'IIIIIIYZYZ': -0.10460420805448783,
 'IIIIIIYZZY': 0.0020603107584759067,
 'IIIIIIZIII': 1.6496131029811243,
 'IIIIIIZIIZ': 0.2515869355056082,
 'IIIII

In [7]:
print(f"n_qubits:  {hamiltonians[mol_key][1]}")

n_qubits:  10


# Get non-contextual H

In [8]:
nonH_guesses = c.greedy_dfs(ham, 10, criterion='weight')

nonH = max(nonH_guesses, key=lambda x:len(x)) # largest nonCon part found by dfs alg

In [9]:
nonH

['IIIIIIIIII',
 'IIIIIIIIIZ',
 'IIIIIIIIZI',
 'IZIIIIIIII',
 'IIIIIIIZII',
 'IIIIIIZIII',
 'IIIZIIIIII',
 'IIZIIIIIII',
 'IIIIIZIIII',
 'IIIIZIIIII',
 'IIIIIIIIZZ',
 'ZIIIIIIIII',
 'ZIIIZZIIII',
 'ZZZIZIZIZI',
 'ZZIZZIIZIZ',
 'IZIIIIIIIZ',
 'IZIIIIIIZI',
 'IIIIZXZZZZ',
 'ZIIIIXZZZZ',
 'IZIZIIIIII',
 'IZZIIIIIII',
 'IZIIIIIZII',
 'IZIIIIZIII',
 'IIIIZZIIII',
 'IZIIIZIIII',
 'IZIIZIIIII',
 'ZZIIIIIIII',
 'ZZIIZZIIII',
 'ZIIIIZIIII',
 'ZIIZZIIZIZ',
 'ZIZIZIZIZI',
 'IIIIIIIZZI',
 'IIIIIIZIIZ',
 'IIIZIIIIZI',
 'IIZIIIIIIZ',
 'IIIZIIIIIZ',
 'IIZIIIIIZI',
 'IIIIIIIZIZ',
 'IIIIIIZIZI',
 'ZIIIZIIIII',
 'ZIIIIIIIIZ',
 'ZIIIZZIIZI',
 'ZIIIIIIIZI',
 'ZIIIZZIIIZ',
 'ZZIZZIIZZZ',
 'ZZZIZIZIZZ',
 'IIIIIZIIZI',
 'IIIIZIIIIZ',
 'IIIIIZIIIZ',
 'IIIIZIIIZI',
 'ZZIZZIIZII',
 'ZZZIZIZIII',
 'IIZZIIIIII',
 'IIIIIIZZII',
 'IIIZIIZIII',
 'IIZIIIIZII',
 'IIIIIZZIII',
 'IIIIZIIZII',
 'ZIIIIIIZII',
 'ZIIIZZZIII',
 'ZZIZZIZZIZ',
 'ZZZIZIZZZI',
 'ZIIZIIIIII',
 'ZIZIZZIIII',
 'IIZZIIZZZZ',
 'IIIZZIIIII',
 'IIZIIZII

Split into:

$$H = H_{c} + H_{nc}$$

In [10]:
nonCon_H = {}
Con_H = {}

for P in ham:
    if P in nonH:
        nonCon_H[P]=ham[P]
    else:
        Con_H[P]=ham[P]
        

## Testing contextuality

In [11]:
print('Is NONcontextual correct:', not c.contextualQ_ham(nonCon_H))
print('Is contextual correct:',c.contextualQ_ham(Con_H))

Is NONcontextual correct: True
Is contextual correct: True


# Classical part of problem!

Take $H_{nc}$ and split into:
- $Z$ = operators that completely comute with all operators in $S$
- $T$ = remaining operators in $S$
    - where $S = Z \cup T$  and $S$ is set of Pauli operators in $H_{nc}$
    
    
- We then split the set $T$ into cliques $C_{1}, C_{2}, ... , C_{|T|}$
    - all ops in a clique commute
    - ops between cliques anti-commute!

In [12]:
bool_flag, Z_list, T_list = c.contextualQ(list(nonCon_H.keys()), verbose=True)

In [13]:
Z_list

['IIIIIIIIII',
 'IIIIIIIIIZ',
 'IIIIIIIIZI',
 'IIIIIIIIZZ',
 'IIIIIIIZII',
 'IIIIIIIZIZ',
 'IIIIIIIZZI',
 'IIIIIIZIII',
 'IIIIIIZIIZ',
 'IIIIIIZIZI',
 'IIIIIIZZII',
 'IIIIZIIIII',
 'IIIIZIIIIZ',
 'IIIIZIIIZI',
 'IIIIZIIZII',
 'IIIIZIZIII',
 'IIIZIIIIII',
 'IIIZIIIIIZ',
 'IIIZIIIIZI',
 'IIIZIIIZII',
 'IIIZIIZIII',
 'IIIZZIIIII',
 'IIZIIIIIII',
 'IIZIIIIIIZ',
 'IIZIIIIIZI',
 'IIZIIIIZII',
 'IIZIIIZIII',
 'IIZIZIIIII',
 'IIZZIIIIII',
 'IIZZIIZZZZ',
 'IZIIIIIIII',
 'IZIIIIIIIZ',
 'IZIIIIIIZI',
 'IZIIIIIZII',
 'IZIIIIZIII',
 'IZIIZIIIII',
 'IZIZIIIIII',
 'IZIZZIIZIZ',
 'IZZIIIIIII',
 'IZZIZIZIZI',
 'ZIIIIIIIII',
 'ZIIIIIIIIZ',
 'ZIIIIIIIZI',
 'ZIIIIIIZII',
 'ZIIIIIZIII',
 'ZIIIZIIIII',
 'ZIIZIIIIII',
 'ZIIZZIIZIZ',
 'ZIZIIIIIII',
 'ZIZIZIZIZI',
 'ZZIIIIIIII',
 'ZZIIZIIZIZ',
 'ZZIIZIZIZI',
 'ZZIZIIIZIZ',
 'ZZIZZIIIIZ',
 'ZZIZZIIZII',
 'ZZIZZIIZIZ',
 'ZZIZZIIZZZ',
 'ZZIZZIZZIZ',
 'ZZZIIIZIZI',
 'ZZZIZIIIZI',
 'ZZZIZIZIII',
 'ZZZIZIZIZI',
 'ZZZIZIZIZZ',
 'ZZZIZIZZZI',
 'ZZZZZIIZIZ',
 'ZZZZZIZI

In [14]:
T_list

['IIIIIXZZZZ',
 'IIIIIZIIII',
 'IIIIIZIIIZ',
 'IIIIIZIIZI',
 'IIIIIZIZII',
 'IIIIIZZIII',
 'IIIIZXIZZZ',
 'IIIIZXZIZZ',
 'IIIIZXZZIZ',
 'IIIIZXZZZI',
 'IIIIZXZZZZ',
 'IIIIZZIIII',
 'IIIZIZIIII',
 'IIIZZXZZZZ',
 'IIZIIZIIII',
 'IIZIZXZZZZ',
 'IZIIIZIIII',
 'IZIIZXZZZZ',
 'IZIZIZIZIZ',
 'IZIZZXZIZI',
 'IZZIIZZIZI',
 'IZZIZXIZIZ',
 'ZIIIIXIZZZ',
 'ZIIIIXZIZZ',
 'ZIIIIXZZIZ',
 'ZIIIIXZZZI',
 'ZIIIIXZZZZ',
 'ZIIIIZIIII',
 'ZIIIZXZZZZ',
 'ZIIIZZIIII',
 'ZIIIZZIIIZ',
 'ZIIIZZIIZI',
 'ZIIIZZIZII',
 'ZIIIZZZIII',
 'ZIIZIXZZZZ',
 'ZIIZZZIIII',
 'ZIZIIXZZZZ',
 'ZIZIZZIIII',
 'ZZIIIXZZZZ',
 'ZZIIZZIIII',
 'ZZIZIXZIZI',
 'ZZIZZZIZIZ',
 'ZZZIIXIZIZ',
 'ZZZIZZZIZI']

## Get quasi model

First we define

- $C_{i1}$ = first Pauli in each $C_{i}$ set
- $A_{ij} = C_{ij}C_{1i}$


- $G^{prime} = \{1 P_{i} \;| \; i=1,2,...,|Z| \}$
    - aka all the completely commuting terms with coefficients set to +1!

- We define G to be an independent set of $G^{prime}$
    - where $G \subseteq G^{prime}$


In [15]:
G_list, Ci1_list, all_mappings = c.quasi_model(nonCon_H)

In [16]:
print('non-independent Z list:', Z_list)
print('G (independent) Z list:', G_list)

non-independent Z list: ['IIIIIIIIII', 'IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIIZZ', 'IIIIIIIZII', 'IIIIIIIZIZ', 'IIIIIIIZZI', 'IIIIIIZIII', 'IIIIIIZIIZ', 'IIIIIIZIZI', 'IIIIIIZZII', 'IIIIZIIIII', 'IIIIZIIIIZ', 'IIIIZIIIZI', 'IIIIZIIZII', 'IIIIZIZIII', 'IIIZIIIIII', 'IIIZIIIIIZ', 'IIIZIIIIZI', 'IIIZIIIZII', 'IIIZIIZIII', 'IIIZZIIIII', 'IIZIIIIIII', 'IIZIIIIIIZ', 'IIZIIIIIZI', 'IIZIIIIZII', 'IIZIIIZIII', 'IIZIZIIIII', 'IIZZIIIIII', 'IIZZIIZZZZ', 'IZIIIIIIII', 'IZIIIIIIIZ', 'IZIIIIIIZI', 'IZIIIIIZII', 'IZIIIIZIII', 'IZIIZIIIII', 'IZIZIIIIII', 'IZIZZIIZIZ', 'IZZIIIIIII', 'IZZIZIZIZI', 'ZIIIIIIIII', 'ZIIIIIIIIZ', 'ZIIIIIIIZI', 'ZIIIIIIZII', 'ZIIIIIZIII', 'ZIIIZIIIII', 'ZIIZIIIIII', 'ZIIZZIIZIZ', 'ZIZIIIIIII', 'ZIZIZIZIZI', 'ZZIIIIIIII', 'ZZIIZIIZIZ', 'ZZIIZIZIZI', 'ZZIZIIIZIZ', 'ZZIZZIIIIZ', 'ZZIZZIIZII', 'ZZIZZIIZIZ', 'ZZIZZIIZZZ', 'ZZIZZIZZIZ', 'ZZZIIIZIZI', 'ZZZIZIIIZI', 'ZZZIZIZIII', 'ZZZIZIZIZI', 'ZZZIZIZIZZ', 'ZZZIZIZZZI', 'ZZZZZIIZIZ', 'ZZZZZIZIZI']
G (independent) Z list: ['ZIIIIIIIII'

In [17]:
print('all T list (dependent):', T_list)
print('all Ci1 terms (independent):', Ci1_list)

# T list can be built from Ci1 terms AND some combination of G_list!

all T list (dependent): ['IIIIIXZZZZ', 'IIIIIZIIII', 'IIIIIZIIIZ', 'IIIIIZIIZI', 'IIIIIZIZII', 'IIIIIZZIII', 'IIIIZXIZZZ', 'IIIIZXZIZZ', 'IIIIZXZZIZ', 'IIIIZXZZZI', 'IIIIZXZZZZ', 'IIIIZZIIII', 'IIIZIZIIII', 'IIIZZXZZZZ', 'IIZIIZIIII', 'IIZIZXZZZZ', 'IZIIIZIIII', 'IZIIZXZZZZ', 'IZIZIZIZIZ', 'IZIZZXZIZI', 'IZZIIZZIZI', 'IZZIZXIZIZ', 'ZIIIIXIZZZ', 'ZIIIIXZIZZ', 'ZIIIIXZZIZ', 'ZIIIIXZZZI', 'ZIIIIXZZZZ', 'ZIIIIZIIII', 'ZIIIZXZZZZ', 'ZIIIZZIIII', 'ZIIIZZIIIZ', 'ZIIIZZIIZI', 'ZIIIZZIZII', 'ZIIIZZZIII', 'ZIIZIXZZZZ', 'ZIIZZZIIII', 'ZIZIIXZZZZ', 'ZIZIZZIIII', 'ZZIIIXZZZZ', 'ZZIIZZIIII', 'ZZIZIXZIZI', 'ZZIZZZIZIZ', 'ZZZIIXIZIZ', 'ZZZIZZZIZI']
all Ci1 terms (independent): ['IIIIIZIIII', 'IIIIIXZZZZ']


$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

In [18]:
# Assemble all the mappings from terms in the Hamiltonian to their products in R:
all_mappings

{'IIIIIIIIII': [[], [], 1],
 'IIIIIIIIIZ': [['IIIIIIIIIZ'], [], 1],
 'IIIIIIIIZI': [['IIIIIIIIZI'], [], 1],
 'IIIIIIIIZZ': [['IIIIIIIIZI', 'IIIIIIIIIZ'], [], 1],
 'IIIIIIIZII': [['IIIIIIIZII'], [], 1],
 'IIIIIIIZIZ': [['IIIIIIIZII', 'IIIIIIIIIZ'], [], 1],
 'IIIIIIIZZI': [['IIIIIIIZII', 'IIIIIIIIZI'], [], 1],
 'IIIIIIZIII': [['IIIIIIZIII'], [], 1],
 'IIIIIIZIIZ': [['IIIIIIZIII', 'IIIIIIIIIZ'], [], 1],
 'IIIIIIZIZI': [['IIIIIIZIII', 'IIIIIIIIZI'], [], 1],
 'IIIIIIZZII': [['IIIIIIZIII', 'IIIIIIIZII'], [], 1],
 'IIIIIXZZZZ': [[], ['IIIIIXZZZZ'], 1],
 'IIIIIZIIII': [[], ['IIIIIZIIII'], 1],
 'IIIIIZIIIZ': [['IIIIIIIIIZ'], ['IIIIIZIIII'], 1],
 'IIIIIZIIZI': [['IIIIIIIIZI'], ['IIIIIZIIII'], 1],
 'IIIIIZIZII': [['IIIIIIIZII'], ['IIIIIZIIII'], 1],
 'IIIIIZZIII': [['IIIIIIZIII'], ['IIIIIZIIII'], 1],
 'IIIIZIIIII': [['IIIIZIIIII'], [], 1],
 'IIIIZIIIIZ': [['IIIIZIIIII', 'IIIIIIIIIZ'], [], 1],
 'IIIIZIIIZI': [['IIIIZIIIII', 'IIIIIIIIZI'], [], 1],
 'IIIIZIIZII': [['IIIIZIIIII', 'IIIIIIIZII'], [], 1]

Overall $R$ is basically reduced non-contextual set
- where everything in original non-contextual set can be found by **inference!**

# Function form

$$R = G \cup \{ C_{i1} \;| \; i=1,2,...,N \}$$

- note q to do with $G$
- note r to do with $C_{i1}$

In [19]:
model = [G_list, Ci1_list, all_mappings]

fn_form = c.energy_function_form(nonCon_H, model)

# returns [
#            denstion of q,
#            dimension of r,
#            [coeff, indices of q's, indices of r's, term in Hamiltonian]
#         ]


In [20]:
fn_form

[9,
 2,
 [[-55.226577416214326, [], [], 'IIIIIIIIII'],
  [12.411397933347452, [8], [], 'IIIIIIIIIZ'],
  [12.411397933347452, [7], [], 'IIIIIIIIZI'],
  [1.1862777224861802, [7, 8], [], 'IIIIIIIIZZ'],
  [1.6496131029811243, [6], [], 'IIIIIIIZII'],
  [0.23693603049649825, [6, 8], [], 'IIIIIIIZIZ'],
  [0.2515869355056082, [6, 7], [], 'IIIIIIIZZI'],
  [1.6496131029811243, [5], [], 'IIIIIIZIII'],
  [0.2515869355056082, [5, 8], [], 'IIIIIIZIIZ'],
  [0.23693603049649825, [5, 7], [], 'IIIIIIZIZI'],
  [0.1817943337373835, [5, 6], [], 'IIIIIIZZII'],
  [-5.791003103036013e-05, [], [1], 'IIIIIXZZZZ'],
  [1.1909223847985508, [], [0], 'IIIIIZIIII'],
  [0.1959701332597126, [8], [0], 'IIIIIZIIIZ'],
  [0.19873710487294718, [7], [0], 'IIIIIZIIZI'],
  [0.12487996093473576, [6], [0], 'IIIIIZIZII'],
  [0.16016822022287117, [5], [0], 'IIIIIZZIII'],
  [1.1909223847985508, [4], [], 'IIIIZIIIII'],
  [0.19873710487294718, [4, 8], [], 'IIIIZIIIIZ'],
  [0.1959701332597126, [4, 7], [], 'IIIIZIIIZI'],
  [0.160168220

In [21]:
Energy_function = c.energy_function(fn_form)

In [22]:
import random

### now for the q terms we only have +1 or -1 assignment!
q_variables = [random.choice([1,-1]) for _ in range(fn_form[0])]


### r variables is anything that makes up unit vector!
r_variables = c.angular(np.arange(0,2*np.pi, fn_form[1]))
r_variables

(1.0, -0.0, -0.0, -0.0, 0.0)

In [23]:
 Energy_function(*q_variables,*r_variables)

-76.34911738934971

find_gs_nonconfunction optimizes above steps by:
1. brute forcing all choices of ```q_variables```
    - ```itertools.product([1,-1],repeat=fn_form[0])```
2. optimizing over ```r_variables``` (in code ```x```)
    - using SciPy optimizer!

In [24]:
model = [G_list, Ci1_list, all_mappings]

lowest_eigenvalue, ground_state_params, model_copy, fn_form_copy,  = c.find_gs_noncon(nonCon_H,
               method = 'differential_evolution',
               model=model,
               fn_form=fn_form) # returns:  best + [model, fn_form]

print(lowest_eigenvalue)
print(ground_state_params)

-83.87422390061548
[[1, -1, -1, -1, -1, -1, -1, -1, -1], [-0.999999999999984, -1.7850620613845431e-07]]


In [25]:
## check
Energy_function(*ground_state_params[0],*ground_state_params[1]) == lowest_eigenvalue

True

# Now need to rotate Hamiltonian!

We now have non contextual ground state: $(\vec{q}, \vec{r})$

In [26]:
ground_state_params

[[1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-0.999999999999984, -1.7850620613845431e-07]]

We can use this result - ground state of $H_{nc}$ -  as a classical estiamte of our ground state of the full Hamiltonian ($H = H_{c} + H_{nc}$)

However we can also obtain a quantum correction using $H_{c}$

By minimizing theenergy of the remaining terms in the Hamiltonian over the quantum states that are **consistent with the noncon-textual ground state**.

To do this we first rotate each $G_{j}$ and $\mathcal{A} = \sum_{i=1}^{N} r_{i}A_{i}$:

In [27]:
model = [G_list, Ci1_list, all_mappings]

print(G_list) # G_j terms!
print(Ci1_list) # mathcal(A)

['ZIIIIIIIII', 'IZIIIIIIII', 'IIZIIIIIII', 'IIIZIIIIII', 'IIIIZIIIII', 'IIIIIIZIII', 'IIIIIIIZII', 'IIIIIIIIZI', 'IIIIIIIIIZ']
['IIIIIZIIII', 'IIIIIXZZZZ']


to SINGLE QUBIT pauli Z operators!

- to map the operators in $G$ to single qubit Pauli operators, we use $\frac{\pi}{2}$ rotations!

- note $\mathcal{A}$ is an anti-commuting set... therefore we can use $N-1$ rotations as in unitary partitioning's sequence of rotations to do this!
    - $R^{\dagger}\mathcal{A} R = \text{single Pauli op}$

# Rotate full Hamiltonian to basis with diagonal noncontextual generators!

function ```diagonalize_epistemic```:
1. first if else statement:
    - if cliques present:
        - first maps A to single Pauli operator (if cliques present)
        - then rotates to diagonlize G union with single Pauli opator of A (hence GuA name!)
    - else if NO cliques present:
        - gets rotations to diagonlize G
        
     - these rotations make up GuA term in code!
2. NEXT code loops over terms in GuA (denoted as g in code)
    - if g is not a single qubit $Z$:
        - code generates code to rotate operator to make g diagonal (rotations)
        - then constructs map of g to single Z (J rotation)
    - Note R is applied to GuA
    
    
#########
- Note rotations are given in Appendix A of https://arxiv.org/pdf/2011.10027.pdf
    - First code checks if g op in GuA is diagonal
        - if so then needs to apply "K" rotation (involving $Y$ and $I$ operators (see pg 11 top) to make it NOT diagononal
    - now operator will be diagnoal!
    - next generate "J" rotation
        - turns non-diagonal operator into a single qubit $Z$ operator!

In [28]:
# Get sequence of rotations requried to diagonalize the generators for the noncontextual ground state!

Rotations_list, diagonalized_generators_GuA, eigen_vals_nonC_ground_state_GuA_ops = c.diagonalize_epistemic(model,
                                                                                                            fn_form,
                                                                                                            ground_state_params)

Rotations_list

[[3.1415928320959994, 'IIIIIYZZZZ']]

In [29]:
# rotations to map A to single Pauli operator!
Rotations_list

[[3.1415928320959994, 'IIIIIYZZZZ']]

In [30]:
# rotations to diagonlize G
diagonalized_generators_GuA

['ZIIIIIIIII',
 'IZIIIIIIII',
 'IIZIIIIIII',
 'IIIZIIIIII',
 'IIIIZIIIII',
 'IIIIIIZIII',
 'IIIIIIIZII',
 'IIIIIIIIZI',
 'IIIIIIIIIZ',
 'IIIIIZIIII']

In [31]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [32]:
eigen_vals_nonC_ground_state_GuA_ops

array([ 1, -1, -1, -1, -1, -1, -1, -1, -1,  1])

In [33]:
from cs_vqe_with_SeqRot import diagonalize_epistemic_SeqRot

R_sk_OP_list, rotations_dict, GuA = diagonalize_epistemic_SeqRot(model,
                                        fn_form,
                                        ground_state_params)

rotations_dict

{'diag': {'rotations': [],
  'ep_state_trans': array([ 1, -1, -1, -1, -1, -1, -1, -1, -1,  1])},
 'A': {'rotations': [],
  'ep_state_trans': array([ 1, -1, -1, -1, -1, -1, -1, -1, -1,  1])}}

In [34]:
from cs_vqe_with_SeqRot import get_reduced_hamiltonians_SeqRot_improved

order = deepcopy(list(range(hamiltonians[mol_key][1])))
HHH = get_reduced_hamiltonians_SeqRot_improved(ham,
                                               model, 
                                               fn_form,
                                               ground_state_params,
                                               order, 
                                               check_reduction=False)

ValueError: operands could not be broadcast together with shapes (0,) (9,) 

In [ ]:
HHH[4]

In [ ]:
rotations_dict

# NEW LCU method